# Đề tài 1. # 
Động đất Nepal 2015 là một trận động đất mạnh khoảng 7,8 hoặc 8,1 độ xảy ra hồi 11:56 NST vào thứ bảy ngày
25 tháng 4 năm 2015. Sau khi cuộc động đất xảy ra, đã có rất nhiều cuộc điều tra khảo sát thực địa nhằm xác định thiệt hại
của các tòa nhà. Tuy nhiên, tại Nepal có số lượng rất lớn các tòa nhà cũng với đặc tính, điều kiện môi trường xung quanh
khác nhau nên việc xác định mức độ thiệt hại của các tòa nhà là rất khó khăn. Mục tiêu của đề tài này là đề xuất một công
cụ dựa trên Học máy nhằm giúp những người thực hiện khảo sát dễ dàng đánh giá mức độ thiệt hại của các tòa nhà trong
khu vực (Phụ lục).

## Import Library ##

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn library
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score


## Load Data ##

In [20]:
X = pd.read_csv("train_values.csv", index_col="building_id")
y = pd.read_csv("train_labels.csv", index_col="building_id")
X_test = pd.read_csv("test_values.csv", index_col="building_id")


In [22]:
X[(["land_surface_condition","foundation_type"])]

,land_surface_condition,foundation_type
building_id,,
802906,t,r
28830,o,r
94947,t,r
590882,t,r
201944,t,r
...,...,...
688636,n,r
669485,t,r
602512,t,r


## Preprocessing ##

In [23]:
y['damage_grade'] = y['damage_grade'].replace({1: 0, 2: 1, 3: 2})
#Encode the dataset
X = pd.get_dummies(X, dtype = int)
print(X.columns)



Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land

In [24]:
X[(["land_surface_condition_n","land_surface_condition_o","land_surface_condition_t","foundation_type_h","foundation_type_i","foundation_type_r"])]

,land_surface_condition_n,land_surface_condition_o,land_surface_condition_t,foundation_type_h,foundation_type_i,foundation_type_r
building_id,,,,,,
802906,0,0,1,0,0,1
28830,0,1,0,0,0,1
94947,0,0,1,0,0,1
590882,0,0,1,0,0,1
201944,0,0,1,0,0,1
...,...,...,...,...,...,...
688636,1,0,0,0,0,1
669485,0,0,1,0,0,1
602512,0,0,1,0,0,1


## Gauss ##

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y , test_size=0.2, random_state=40)


In [6]:
continuous_cols = np.array(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families'])

binary_cols = X_train[X_train.columns.difference(continuous_cols)].columns.to_numpy()

Code theo công thức

In [7]:
class GaussianNBClassifier(object):
    def __init__(self):
        pass

    def fit(self, X, y):
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.model = np.array([np.c_[np.mean(i, axis=0), np.std(i, axis=0)] for i in separated])
        return self

    def _prob(self, x, mean, std):
        exponent = np.exp(- ((x - mean)**2 / (2 * std**2)))
        return np.log(exponent / (np.sqrt(2 * np.pi) * std))

    def predict_log_proba(self, X):
        return [[sum(self._prob(i, *s) for s, i in zip(summaries, x))
                for summaries in self.model] for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)

    def score(self, X, y):
        return sum(self.predict(X) == y) / len(y)

Code tự tạo

In [8]:
gnb = GaussianNBClassifier()
gnb.fit(X_train.to_numpy()[:,:7], y_train["damage_grade"])
#print(gnb.predict_(X_valid[:3]))
y_pred_GNB = gnb.predict(X_valid.to_numpy()[:,:7])

acc_GNB = accuracy_score(y_valid, y_pred_GNB)
print(acc_GNB)
print(confusion_matrix(y_valid, y_pred_GNB))

0.38990042401335356
[[ 2337   708  1937]
 [ 5382  4895 19368]
 [ 2468  1936 13090]]


Thư viện sklearn

In [9]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train[continuous_cols], y_train)
y_pred_GNB = gnb.predict(X_valid[continuous_cols])

print("Accuracy Gauss Naive Baiyes :", accuracy_score(y_valid, y_pred_GNB))
print(classification_report(y_valid, y_pred_GNB))


print(confusion_matrix(y_valid, y_pred_GNB))

Accuracy Gauss Naive Baiyes : 0.5629592678574855
              precision    recall  f1-score   support

           0       0.29      0.14      0.18      4982
           1       0.58      0.94      0.71     29645
           2       0.55      0.05      0.10     17494

    accuracy                           0.56     52121
   macro avg       0.47      0.37      0.33     52121
weighted avg       0.54      0.56      0.46     52121

[[  675  4257    50]
 [ 1187 27745   713]
 [  505 16067   922]]


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Xây dựng Bernoulli

In [10]:
class BernoulliNB(object):
    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X, y):
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        smoothing = 2 * self.alpha
        n_doc = np.array([len(i) + smoothing for i in separated])
        self.feature_prob_ = count / n_doc[np.newaxis].T
        return self

    def predict_log_proba(self, X):
        return [(np.log(self.feature_prob_) * x + np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        return np.argmax(self.predict_log_proba(X), axis=1)

Code tự tạo

In [11]:

mnb = BernoulliNB()
mnb.fit(X_train.to_numpy()[:,10:], y_train["damage_grade"])
y_pred_MNB = mnb.predict(X_valid.to_numpy()[:,10:])
acc_MNB = accuracy_score(y_valid, y_pred_MNB)
print(acc_MNB)
print(precision_score(y_valid, y_pred_MNB, average="weighted"))
print(classification_report(y_valid, y_pred_MNB))

print(confusion_matrix(y_valid, y_pred_MNB))

/var/folders/r5/97p5_1zs7w7gdlq9bjgm3wd80000gn/T/ipykernel_662/483067321.py:16: RuntimeWarning: invalid value encountered in log
  return [(np.log(self.feature_prob_) * x + np.log(1 - self.feature_prob_) * np.abs(x - 1)


0.3356420636595614
0.11265559489764906
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4982
           1       0.00      0.00      0.00     29645
           2       0.34      1.00      0.50     17494

    accuracy                           0.34     52121
   macro avg       0.11      0.33      0.17     52121
weighted avg       0.11      0.34      0.17     52121

[[    0     0  4982]
 [    0     0 29645]
 [    0     0 17494]]


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/anaconda3/lib/python3.11

Code thư viện

In [12]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
bnb.fit(X_train[binary_cols],  y_train)
y_pred_BNB= bnb.predict(X_valid[binary_cols])

print("Accuracy Bernoulli Naive Baiyes :", accuracy_score(y_valid, y_pred_BNB))
print(classification_report(y_valid, y_pred_BNB))

print(confusion_matrix(y_valid, y_pred_BNB))

Accuracy Bernoulli Naive Baiyes : 0.49941482319986186


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.44      0.55      0.49      4982
           1       0.62      0.41      0.50     29645
           2       0.42      0.64      0.51     17494

    accuracy                           0.50     52121
   macro avg       0.49      0.53      0.50     52121
weighted avg       0.54      0.50      0.50     52121

[[ 2740  1331   911]
 [ 3198 12174 14273]
 [  359  6019 11116]]


In [13]:
probs = gnb.predict_proba(X_valid[continuous_cols])* bnb.predict_proba(X_valid[binary_cols])
probs = np.divide(probs, np.sum(probs, axis=1)[:,np.newaxis])
y_pred = np.argmax(probs, axis=1)
print("Accuracy Mixed Naive Baiyes :", accuracy_score(y_valid, y_pred))
print(classification_report(y_valid, y_pred))
print(confusion_matrix(y_valid, y_pred))


Accuracy Mixed Naive Baiyes : 0.5625563592409969
              precision    recall  f1-score   support

           0       0.43      0.48      0.46      4982
           1       0.62      0.68      0.65     29645
           2       0.49      0.39      0.44     17494

    accuracy                           0.56     52121
   macro avg       0.51      0.52      0.51     52121
weighted avg       0.56      0.56      0.56     52121

[[ 2411  2204   367]
 [ 2795 20040  6810]
 [  385 10239  6870]]
